## CLIPSPY tutorial - production rule programming

In [1]:
## Set up the CLIPS environment
import clips
env = clips.Environment()

### The Magic Squares program from Riggs 2000

In [30]:
# Sets up a Magic Square per figure 1
env.clear()
env.build('(defglobal ?*log* = (format nil "INFO:%n"))')
env.build("""
(deffacts init
 (element 1 1 8)
 (element 1 2 1)
 (element 1 3 6)
 (element 2 1 3)
 (element 2 2 5)
 (element 2 3 7)
 (element 3 1 4)
 (element 3 2 9)
 (element 3 3 2)
 (sum row 1 0)
 (sum row 2 0)
 (sum row 3 0)
 (sum col 1 0)
 (sum col 2 0)
 (sum col 3 0)
 (sum diag 1 0)
 (sum diag 2 0)
 (size 3)
)"""
)

In [31]:
env.reset()
for fact in env.facts():
    print(fact)

(initial-fact)
(element 1 1 8)
(element 1 2 1)
(element 1 3 6)
(element 2 1 3)
(element 2 2 5)
(element 2 3 7)
(element 3 1 4)
(element 3 2 9)
(element 3 3 2)
f-10    (sum row 1 0)
f-11    (sum row 2 0)
f-12    (sum row 3 0)
f-13    (sum col 1 0)
f-14    (sum col 2 0)
f-15    (sum col 3 0)
f-16    (sum diag 1 0)
f-17    (sum diag 2 0)
f-18    (size 3)


In [32]:
env.build("""
(defrule make-addends
 (declare (salience 100)) ;; salience > default
 (size ?size)
 (element ?r ?c ?v)
  =>
 (assert 
   (addend row ?r ?v)
   (addend col ?c ?v))
 (if (= ?r ?c) ;; on diagl?
   then (assert (addend diag 1 ?v)))
 (if (= (+ ?r ?c) (+ 1 ?size)) ;; on diag2?
   then (assert (addend diag 2 ?v)))) 
""")

In [33]:
env.build("""
(defrule addem
 (declare (salience 100))
 ?fl <- (addend ?type ?inx ?val)
 ?f2 <- (sum ?type ?inx ?sum)
 =>
 (retract ?fl ?f2) 
 (assert (sum ?type ?inx (+ ?val ?sum)))
 (bind ?*log* 
   (str-cat ?*log*
     (format nil "type: %s; val: %d; sum: %d%n" ?type ?val ?sum)
   )
 )
)
""")

In [34]:
env.build("""
(defrule not-magic
 (sum ? ? ?s)
 (sum ? ? ?r&~?s) ;; i.e. ?s <> ?r
 =>
 (bind ?*log* 
   (str-cat ?*log*
     (format nil "Not Magic.%n")
    )
 )
 (halt)
)
""")

env.build("""
(defrule magic
 (sum row 1 ?val)
 (not (sum ? ? ?v2&~?val)) ;; i.e. not ?v2 <> val
 =>
 (bind ?*log* 
   (str-cat ?*log*
     (format nil "Magic!!%n")
   )
  )
) 
""")

In [35]:
env.run()
v = env.find_global('log')
print(v.value)
for fact in env.facts():
    print(fact)

INFO:
type: diag; val: 2; sum: 0
type: col; val: 2; sum: 0
type: row; val: 2; sum: 0
type: col; val: 9; sum: 0
type: row; val: 9; sum: 2
type: diag; val: 4; sum: 0
type: col; val: 4; sum: 0
type: row; val: 4; sum: 11
type: col; val: 7; sum: 2
type: row; val: 7; sum: 0
type: diag; val: 5; sum: 4
type: diag; val: 5; sum: 2
type: col; val: 5; sum: 9
type: row; val: 5; sum: 7
type: col; val: 3; sum: 4
type: row; val: 3; sum: 12
type: diag; val: 6; sum: 9
type: col; val: 6; sum: 9
type: row; val: 6; sum: 0
type: col; val: 1; sum: 14
type: row; val: 1; sum: 6
type: diag; val: 8; sum: 7
type: col; val: 8; sum: 7
type: row; val: 8; sum: 7
Magic!!

(initial-fact)
(element 1 1 8)
(element 1 2 1)
(element 1 3 6)
(element 2 1 3)
(element 2 2 5)
(element 2 3 7)
(element 3 1 4)
(element 3 2 9)
(element 3 3 2)
f-18    (size 3)
f-34    (sum row 3 15)
f-50    (sum row 2 15)
f-54    (sum diag 2 15)
f-55    (sum col 3 15)
f-59    (sum col 2 15)
f-64    (sum diag 1 15)
f-65    (sum col 1 15)
f-66    (sum 

### Dining Philosophers from Riggs 2000

In [36]:
env.clear()
env.build('(defglobal ?*log* = (format nil "INFO:%n"))')
env.build("""
(deffacts init 
 (p 1 uses 5 1)
 (p 2 uses 1 2)
 (p 3 uses 2 3)
 (p 4 uses 3 4)
 (p 5 uses 4 5)
 (fork 1 avail)
 (fork 2 avail)
 (fork 3 avail)
 (fork 4 avail)
 (fork 5 avail)
 (p 1 thinking)
 (p 2 thinking)
 (p 3 thinking)
 (p 4 thinking)
 (p 5 thinking)
)"""
)
env.reset()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 1 avail)
(fork 2 avail)
(fork 3 avail)
(fork 4 avail)
f-10    (fork 5 avail)
f-11    (p 1 thinking)
f-12    (p 2 thinking)
f-13    (p 3 thinking)
f-14    (p 4 thinking)
f-15    (p 5 thinking)


In [37]:
env.build("""
(defrule eat ;; hungry -> eat
  ?a<-(active ?n) ;; serialization
  ?p<-(p ?n hungry)
      (p ?n uses ?left ?right)
  ?f<-(fork ?right avail)
 =>
 (retract ?a ?p ?f)
 (assert (p ?n eats)   ;; eat
    (fork ?right ?n))  ;; 2nd fork taken
)
""")

env.build("""
(defrule hunger  ;; thinking -> hungry
  ?a<-(active ?n) ;; serialization
  ?p<-(p ?n thinking)
      (p ?n uses ?left ?right)
  ?f<-(fork ?left avail)
 =>
 (retract ?a ?p ?f)
 (assert (p ?n hungry)     ;; hungry
         (fork ?left ?n))  ;; 1st fork taken
)
""")

env.build("""
(defrule think ;; eat -> thinking
 ?a<-(active ?n) ;; serialization
 ?p<-(p ?n eats)
     (p ?n uses ?left ?right)
 ?f1<-(fork ?left ?n)
 ?f2<-(fork ?right ?n)
 =>
 (retract ?a ?p ?f1 ?f2 )
 (assert (p ?n thinking)         ;; thinking
           (fork ?left avail)    ;; fork 1 returned
           (fork ?right avail))  ;; fork 2 returned
)
""")

env.build("""
(defrule dont-eat   ;; hungry -> not eat
 ?a<-(active ?n)    ;; serialization
 ?p<-(p ?n hungry)
     (p ?n uses ?left ?right)
 ?f<- (fork ?left ?n)
      (not (fork ?right avail)) ;; can't get right
  =>
 (retract ?a ?p ?f)
 (assert (p ?n thinking)
         (fork ?left down))  ;; release left
)
""")


In [38]:
env.run()
v = env.find_global('log')
print(v.value)
for fact in env.facts():
    print(fact)

INFO:

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 1 avail)
(fork 2 avail)
(fork 3 avail)
(fork 4 avail)
f-10    (fork 5 avail)
f-11    (p 1 thinking)
f-12    (p 2 thinking)
f-13    (p 3 thinking)
f-14    (p 4 thinking)
f-15    (p 5 thinking)


In [39]:
env.assert_string('(active 1)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 1 avail)
(fork 2 avail)
(fork 3 avail)
(fork 4 avail)
f-12    (p 2 thinking)
f-13    (p 3 thinking)
f-14    (p 4 thinking)
f-15    (p 5 thinking)
f-17    (p 1 hungry)
f-18    (fork 5 1)


In [40]:
env.assert_string('(active 2)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 2 avail)
(fork 3 avail)
(fork 4 avail)
f-13    (p 3 thinking)
f-14    (p 4 thinking)
f-15    (p 5 thinking)
f-17    (p 1 hungry)
f-18    (fork 5 1)
f-20    (p 2 hungry)
f-21    (fork 1 2)


In [41]:
env.assert_string('(active 3)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 3 avail)
(fork 4 avail)
f-14    (p 4 thinking)
f-15    (p 5 thinking)
f-17    (p 1 hungry)
f-18    (fork 5 1)
f-20    (p 2 hungry)
f-21    (fork 1 2)
f-23    (p 3 hungry)
f-24    (fork 2 3)


In [42]:
env.assert_string('(active 4)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
(fork 4 avail)
f-15    (p 5 thinking)
f-17    (p 1 hungry)
f-18    (fork 5 1)
f-20    (p 2 hungry)
f-21    (fork 1 2)
f-23    (p 3 hungry)
f-24    (fork 2 3)
f-26    (p 4 hungry)
f-27    (fork 3 4)


In [43]:
env.assert_string('(active 5)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
f-17    (p 1 hungry)
f-18    (fork 5 1)
f-20    (p 2 hungry)
f-21    (fork 1 2)
f-23    (p 3 hungry)
f-24    (fork 2 3)
f-26    (p 4 hungry)
f-27    (fork 3 4)
f-29    (p 5 hungry)
f-30    (fork 4 5)


In [44]:
env.assert_string('(active 2)')
env.run()
for fact in env.facts():
    print(fact)

(initial-fact)
(p 1 uses 5 1)
(p 2 uses 1 2)
(p 3 uses 2 3)
(p 4 uses 3 4)
(p 5 uses 4 5)
f-17    (p 1 hungry)
f-18    (fork 5 1)
f-23    (p 3 hungry)
f-24    (fork 2 3)
f-26    (p 4 hungry)
f-27    (fork 3 4)
f-29    (p 5 hungry)
f-30    (fork 4 5)
f-32    (p 2 thinking)
f-33    (fork 1 down)
